<a href="https://colab.research.google.com/github/brook-miller/2023mbai417/blob/main/4-class/scent_iot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
#@title Standard imports - we'll use in most EDA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

from datetime import datetime, timedelta
from dateutil.parser import parse
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [30]:
#@title Extra imports for this lab

!pip install faker
from faker import Faker
fake = Faker()
import random
import json


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.7 MB/s eta 0:00:00


In [31]:
#@title ScentData class models the type of data received from IoT devices
class ScentData:
    #our sample api / scent data class
    def __init__(self, i):
        self.latitude = (random.random() * 180) - 90
        self.longitude = (random.random() * 360) - 180
        self.location = fake.address()
        self.company = fake.company()
        self.sqi = random.random()
        self.air_temp = random.random()
        self.fill_level = random.random()
        self.id = i

In [32]:
#@title Build sample data and save it to 3 file formats
items = [ScentData(i) for i in range(0,1000)]

df = pd.DataFrame([vars(f) for f in items])
df.to_parquet("data.parquet")
df.to_csv("data.csv", sep=',', encoding='utf-8')

json_string = json.dumps([item.__dict__ for item in items], sort_keys=True, indent=4)

with open('data.json', 'w') as file:
    file.write(json_string)

In [33]:
#@title Check file sizes

import os

files = ["data.json", "data.csv", "data.parquet"]

for file in files:
  file_size = os.path.getsize(file)
  print(f"{file} size is : {file_size} bytes")

data.json size is : 337495 bytes
data.csv size is : 168167 bytes
data.parquet size is : 105579 bytes


In [34]:
#@title these assumptions are all given no need to change them
device_cost = 200
cellular_per_device_month = 0.65
cellular_month_per_MB = 0.35
kafka_per_hour = 0.75
kafka_per_GB = 0.20
s3_per_GB = 0.023

In [37]:
#@title these are variables, determine the size in byte by dividing the data.json size by 1000

bytes_per_json = 1030 #@param {type:"slider", min:1, max:10000, step:1}

num_devices = 5020 #@param {type:"slider", min:1, max:10000, step:1}

num_readings = 46 #@param {type:"slider", min:1, max:1440, step:1}

In [38]:
#@title calculating fixed and monthly costs
fixed = 200 * num_devices
monthly_bytes = num_readings * 30 * bytes_per_json * num_devices
bytes_per_MB = 2**20
monthlyMB = monthly_bytes / bytes_per_MB
month = 1
monthly = (
          cellular_per_device_month * num_devices + 
          cellular_month_per_MB * .35 * monthlyMB +
          kafka_per_hour * 720 +
          kafka_per_GB * monthlyMB / 2**10 + #1024GB per MB
          s3_per_GB * month * monthlyMB / 2**10
)
monthly

4638.079037364572

In [ ]:
#@title the above monthly estimate didn't take into account that the s3 data would be in compressed parquet format. Can you add that in?



In [ ]:
#@title now turn your monthly cost builder into a function and build data for each of 36 months (month goes from 1 to 36)
